In [2]:
# Notebook which demonstrates concept of Broadcasting
# Ex. A = np.dot(W.T, X) + b
# b becomes a 1*m vector consisting of all values as b

import numpy as np

A = np.array([[56.0, 0.0, 4.4, 68.0],
              [1.2, 104.0, 52.0, 8.0],
              [1.8, 135.0, 99.0, 0.9]])

A

array([[ 56. ,   0. ,   4.4,  68. ],
       [  1.2, 104. ,  52. ,   8. ],
       [  1.8, 135. ,  99. ,   0.9]])

In [6]:
total_cal = A.sum(axis=0)
print(total_cal.shape, total_cal)

(4,) [ 59.  239.  155.4  76.9]


In [8]:
percentage_A = 100 * A/total_cal.reshape(1,4)
percentage_A

array([[94.91525424,  0.        ,  2.83140283, 88.42652796],
       [ 2.03389831, 43.51464435, 33.46203346, 10.40312094],
       [ 3.05084746, 56.48535565, 63.70656371,  1.17035111]])